In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 
              'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 
              'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 
              'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
training = pd.read_csv('kc_house_train_data.csv')
test = pd.read_csv('kc_house_test_data.csv')
data = pd.read_csv('kc_house_data2.csv')

In [2]:
def get_numpy_data(data_frame, features:list, output:str):
    data['constant'] = 1
    fea = ['constant']
    fea.extend(features)
    x_matrix = data[fea].to_numpy()
    y_vector = data[output].to_numpy()
    return x_matrix, y_vector

In [3]:
def predict_output(x_matrix, weights):
    return x_matrix @ weights

In [4]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized = feature_matrix / norms
    return normalized, norms

# effect of l1-penalty

In [5]:
feature_matrix, output = get_numpy_data(data, ['sqft_living','bedrooms'], 'price')
normalized_feature_matrix, norms = normalize_features(feature_matrix)
ini_weights = np.array([1.0, 4.0, 1.0])
pred = predict_output(normalized_feature_matrix, ini_weights)
rho = []
for i in range(len(ini_weights)):
    rho.append((output - pred + ini_weights[i]*normalized_feature_matrix[:,i]) @ normalized_feature_matrix[:,i])
print(rho)

[79400300.01452291, 87939470.82325175, 80966698.66623946]


In [6]:
lam = [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]
for l in lam:
    if abs(rho[1]) > l/2 and abs(rho[2]) <= l/2:
        print(l)
for l in lam:
    if abs(rho[1]) <= l/2 and abs(rho[2]) <= l/2:
        print(l)

164000000.0
173000000.0
190000000.0
230000000.0


# single coordinate descent step

In [7]:
def lasso_coo_descent_step(j, feature_matrix, output, weights, l1_penalty):
    """
    update j-th weight, corresponding to j-th column of feature, i.e. [:,j]
    """
    pred = predict_output(feature_matrix, weights)
    rhoj = (output - pred + weights[j]*feature_matrix[:,j]) @ feature_matrix[:,j]
    if j == 0:
        weightj = rhoj
    else:
        abs_value = max(abs(rhoj) - l1_penalty/2, 0)
        weightj = abs_value if rhoj > 0 else -abs_value
    return weightj

In [8]:
import math
lasso_coo_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.4255588466910251

# coordinate descent cycle

In [15]:
def lasso_coo_descent_cycle(feature_matrix, output, ini_weights, l1_penalty, tol):
    n = len(ini_weights)
    weights = ini_weights
    while True:
        change = np.zeros(n)
        for j in range(n):
            new_weight = lasso_coo_descent_step(j, feature_matrix, output, weights, l1_penalty)
            change[j] = abs(new_weight - weights[j])
            weights[j] = new_weight
        if np.max(change) <= tol:
            break
    return weights

In [19]:
coefs = lasso_coo_descent_cycle(normalized_feature_matrix, output, np.zeros(3), 1e7, 1.0)
pred = predict_output(normalized_feature_matrix, coefs)
print('%.2e'%(np.linalg.norm(output - pred) ** 2))
print(coefs)

1.63e+15
[21624997.95951909 63157247.20788956        0.        ]


# Evaluating lasso

In [24]:
feature_list = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 
                          'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
feature_matrix, output = get_numpy_data(training, feature_list, 'price')
normalized_feature_matrix, norms = normalize_features(feature_matrix)

In [26]:
weight1e7 = lasso_coo_descent_cycle(normalized_feature_matrix, output, np.zeros(len(norms)), 1e7, 1.0)

In [31]:
feature_list.insert(0,'constant')
for feature, coef in zip(feature_list,weight1e7):
    print(feature, coef)

constant 24964802.80942066
bedrooms 0.0
bathrooms 0.0
sqft_living 56397533.51865719
sqft_lot 0.0
floors 0.0
waterfront 3689656.5671645254
view 8630251.047030006
condition 0.0
grade 0.0
sqft_above 0.0
sqft_basement 0.0
yr_built 0.0
yr_renovated 0.0


In [33]:
weight1e8 = lasso_coo_descent_cycle(normalized_feature_matrix, output, np.zeros(len(norms)), 1e8, 1.0)
for feature, coef in zip(feature_list,weight1e8):
    print(feature, coef)

constant 79400304.63764462
bedrooms 0.0
bathrooms 0.0
sqft_living 0.0
sqft_lot 0.0
floors 0.0
waterfront 0.0
view 0.0
condition 0.0
grade 0.0
sqft_above 0.0
sqft_basement 0.0
yr_built 0.0
yr_renovated 0.0


In [35]:
weight1e4 = lasso_coo_descent_cycle(normalized_feature_matrix, output, np.zeros(len(norms)), 1e4, 5e5)
for feature, coef in zip(feature_list,weight1e4):
    print(feature, coef)

constant 88587182.82932705
bedrooms -21715665.19372022
bathrooms 12996574.12832279
sqft_living 107727552.02386698
sqft_lot -2423452.287012554
floors -3628590.8002165537
waterfront 6944215.974362667
view 8272975.157013026
condition 6733836.673536284
grade 18656166.722223114
sqft_above -21903168.087684356
sqft_basement -6998131.477065349
yr_built -99896102.00648671
yr_renovated 3274352.3859126647


In [40]:
feature_list.remove('constant')
test_feature_matrix, test_output = get_numpy_data(test, feature_list, 'price')
lst = [weight1e4, weight1e7, weight1e8]
for w in lst:
    original_weights = w / norms
    pred = predict_output(test_feature_matrix, original_weights)
    rss = np.linalg.norm(test_output - pred) ** 2
    print(rss)


1211008049702451.2
1476340533220514.5
2912916761921299.5
